<a href="https://colab.research.google.com/github/nikhilnair31/MIS284N-UD-YT-Shorts-Generator/blob/main/MIS284N_Unstructured_Data_Test_3B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install openai
!pip install praw
!pip install elevenlabs
!pip install pydub
!pip install ffmpeg-python
!pip install opencv-python-headless moviepy
!pip install replicate
!pip install google-auth google-auth-oauthlib
!pip install --upgrade google-auth

In [9]:
import re
import os
import sys
import json
import time
import numpy as np
import pandas as pd
from datetime import datetime

from google.colab import drive
from IPython.display import Image, display

In [3]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
# Specify the path to the shell script file
script_path = "/content/drive/MyDrive/Colab Notebooks/UD/env_vars.sh"

# Read the shell script file
with open(script_path) as file:
    script_content = file.readlines()

# Extract environment variables and assign them to os.environ
for line in script_content:
    if line.startswith("export "):
        key, value = line[len("export "):].strip().split("=")
        os.environ[key] = value.strip('"')

In [11]:
os.environ['REDDIT_USER_AGENT']

'Test UT'

# Reddit Shorts Generation

## Reddit Scrape

In [12]:
import praw

# Create a Reddit API instance
reddit = praw.Reddit(
    client_id=os.environ['REDDIT_CLIENT_ID'],
    client_secret=os.environ['REDDIT_CLIENT_SECRET'],
    user_agent=os.environ['REDDIT_USER_AGENT']
)

# Get the subreddit object
subreddit = reddit.subreddit("shortscarystories")
# shortscarystories AmItheAsshole

In [13]:
total_media_to_retrieve = 35
results = []
params = {'limit': 7}  # Specify the number of items per request

while len(results) < total_media_to_retrieve:
    remaining_to_retrieve = total_media_to_retrieve - len(results)
    current_limit = min(remaining_to_retrieve, params['limit'])

    batch = list(subreddit.top(limit=current_limit, params=params, time_filter="week"))

    if not batch:
        break

    results.extend(batch)

    params = {'limit': params['limit'], 'after': batch[-1].fullname}

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [15]:
# Create a dictionary to store post data
posts_dict = {}

# Iterate through the top posts
for i, post in enumerate(results):
    posts_dict[i] = {
        "Title": post.title,
        "Content": post.selftext,
        # "URL": post.url,
        "Timestamp": datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S UTC'),
        "Upvotes": post.ups,
        "Total Comments": post.num_comments,
    }

In [16]:
# Create a DataFrame from the dictionary
test_df = pd.DataFrame(posts_dict).T
# Convert "Timestamp" column to datetime
test_df["Timestamp"] = pd.to_datetime(test_df["Timestamp"])
# Check post content length
test_df["Post Length"] = test_df['Content'].apply(len)

test_df.head()

,Title,Content,Timestamp,Upvotes,Total Comments,Post Length
0,I deserved the divorce. But no one deserves wh...,"Alimony bleeds me dry every paycheck, but that...",2023-08-25 18:01:30+00:00,1054,58,2562
1,How to cheat Death,*Cheating Death is easy— People do it unconsci...,2023-08-28 03:13:08+00:00,1015,37,2474
2,My daughter is a bully,The police found me in an apartment on the wro...,2023-08-23 11:32:56+00:00,795,14,2823
3,I ran into my old roommate for the first time ...,"While walking down the street, I ran into my o...",2023-08-23 16:23:17+00:00,733,33,2758
4,My girlfriend says I’m becoming the man of her...,The day I bought DreamSync was the first time ...,2023-08-28 15:55:50+00:00,706,28,2685


In [46]:
min_test_df = test_df.sort_values(by=['Post Length'], ascending=[True])[:7]
min_test_df

,Title,Content,Timestamp,Upvotes,Total Comments,Post Length
29,License to Kill,"*One...*\n\nA methed up woman, screaming something about a demon?\n\n*Two...*\n\nI remember the first time I killed someone. The shock didn't last long, as another man ran at me, screaming incoherently. Never could tell the difference between late stage mental illness or developmental delays...\n\n*Three...*\n\nThird one in ten minutes? Must be heading into the bad part of town.\n\nThey call me in to put down...undesirables.\n\n*Four...*\n\nYou know the types. Hard drug users, the severely mentally disabled, the ones with behavior problems, spoiled people who need to learn the word 'no' the hard way, the works.\n\n*Five...*\n\nRich fucks don't want to get their hands dirty; politicians need the dregs of society to go away in order to get votes. Not that I care, I see it as a win-win-win. The more shots I fire off, the more I get paid. The more shots that I fire off in a short amount of time, means I'm about to eat good that week.\n\n*Six...*\n \nI's an easy job, once the disregard for human life sets in. Lucrative as fuck too. My only advice: pack more ammo than you think you'll need.\n\n*Seven...*\n\nThe roughest parts of my job, though...\n\n*EIGHT! NINE! TEN!*\n\nAre the kids.\n\n*chk-CHK*",2023-08-25 19:27:04+00:00,109,9,1177
23,My Daughter's Brush With Death,"My daughter Lisa used to be quite a happy and friendly child. She used to be the best at her class and respectful towards the elders. All that changed when puberty hit.\n\n...\n\nI remember the day after her thirteenth birthday like it was yesterday. I was sleeping when suddenly her screams pierced my brain. ""Help!"" she cried. I rushed to her room only to find her lying in a pool of blood. She had tried to slit her wrists and regretted it!\n\nI panicked. I knew what I had to do: call the ambulance, follow their orders, and wait untill they arrived, but one thing is knowing and another is saving your daughter's life!\n\nThankfully, she hadn't cut deep enough and didn't die.\n\nI would often try to reason with her, tell her what she'd done was wrong, but she'd only give me that look which seemed to cut right through me and I couldn't break eye contact.\n\n""I hate you and your outdated understanding of psychology! You're the reason I'm not happy, you should do something other than nag me non-stop!"" she told me a while ago.\n\n...\n\nLisa is sixteen now. She's happy once again, but her school achievements are gone, just like her ability to hold an intelligent conversation. But still, I've managed to solve most of the problem with just one needle.",2023-08-24 14:43:57+00:00,145,5,1246
31,Just a little while,"Do you ever just wish you could disappear for a little while? Not like going on a vacation… I can never truly relax on those because there’s always still responsibilities waiting, stacking up while gone, so you come back to more of a mess than you left… But also not in a suicidal type of way, I do want to come back and I love my family and know my kids need me… I just wish I could just not exist for a short period of time and come back without it seeming like I was gone. And you know what? I think my wish came true. A few months ago, I closed my eyes and felt like I took the most peaceful, several hour nap, and when I opened them it had only been a couple minutes. It started out just once a week, but slowly became more frequent until it was a couple times a day. I loved it, I was so much more relaxed, and it didn’t affect my day… until I woke from one yesterday and I was in the hospital… my husband said I had a seizure, the doctors found a mass and they were taking me in for surgery tomorrow. I remember going in for surgery, and going back to my peaceful place, but now I feel like I’ve been here for days, but surely that means it’s only been a couple hours, right? That’s normal for surgery, right? There’s no way I’m stuck here, right?",2023-08-26 14:22:10+00:00,104,13,1255
14,She’ll On

## Seed Content

In [47]:
def preprocess_text(text):
    # Remove newlines and extra spaces
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove special characters and numbers
    # text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove asterisks (*) and backslashes (\)
    text = re.sub(r'[\*\\]', '', text)

    # Remove digits at the beginning of lines
    text = re.sub(r'^\d+\s+', '', text, flags=re.MULTILINE)

    return text

pd.set_option('display.max_colwidth', None)
seed_content = str(min_test_df['Content'].iloc[2])
preprocessed_content = preprocess_text(seed_content)
print(f'original text\n{seed_content}\n\nprocessed text\n{preprocessed_content}')

original text
Do you ever just wish you could disappear for a little while? Not like going on a vacation… I can never truly relax on those because there’s always still responsibilities waiting, stacking up while gone, so you come back to more of a mess than you left… But also not in a suicidal type of way, I do want to come back and I love my family and know my kids need me… I just wish I could just not exist for a short period of time and come back without it seeming like I was gone. And you know what? I think my wish came true. A few months ago, I closed my eyes and felt like I took the most peaceful, several hour nap, and when I opened them it had only been a couple minutes. It started out just once a week, but slowly became more frequent until it was a couple times a day. I loved it, I was so much more relaxed, and it didn’t affect my day… until I woke from one yesterday and I was in the hospital… my husband said I had a seizure, the doctors found a mass and they were taking me in 

## Generating Title and Description

In [48]:
import openai

In [49]:
openai.api_key = os.environ['OPENAI_API_KEY']

In [50]:
def generate_title(prompt, script):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": prompt},
                  {"role": "user", "content": script}],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message['content']

In [51]:
shorts_title = generate_title("Generate a very short title for a YouTube Short based on the script. Make it similar to the examples below. Examples: - Scary things hidden in normal photos Part#28 - This is the scariest video on the internet... - Scary video😳 #shorts", preprocessed_content)
shorts_description = generate_title("Generate a short description for a YouTube Short", preprocessed_content)
print(f'Title: {shorts_title}\nDescription: {shorts_description}\n')

Title: YouTube Short: The Mystery Naps and the Unseen Reality
Description: "Escaping Reality: A YouTube Short about a Woman's Mysterious Disappearing Act"



## Image Prompts

In [52]:
def generate_image_prompts(gpt_type, prompt, script):
    response = openai.ChatCompletion.create(
        model=gpt_type,
        messages=[{"role": "system", "content": prompt},
                  {"role": "user", "content": script}],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    image_prompts_str = response.choices[0].message['content']
    image_prompts_str = re.sub(r'[^\w\s]', '', image_prompts_str)
    image_prompts_str = re.sub(r'[^a-zA-Z\s]', '', image_prompts_str)
    image_prompts = image_prompts_str.lower().split('\n')
    image_prompts = list(map(lambda prompt: f"{prompt.strip()}", image_prompts))

    return image_prompts

In [59]:
gpt_type = "gpt-3.5-turbo"
gpt_prompt = """
  Generate 6 short one liner prompts for DALLE-2/Midjourney/StableDiffusion prompt image generation. The prompt I give will be the script for a video.
  Prompts should be very short but descriptive with no NSFW content.
"""
image_prompts = generate_image_prompts(gpt_type, gpt_prompt, preprocessed_content)
image_prompts

['womans peaceful escape into a timeless realm becomes a haunting reality after surgery',
 'a wish for temporary nonexistence turns into an unnerving journey through time and reality',
 'the allure of disappearing temporarily turns into a disorienting neverending void',
 'seeking solace in moments of oblivion a woman finds herself trapped in a distorted perception of time',
 'escaping the chaos of life a womans tranquil interludes take a dark turn after an unexpected surgery',
 'the desire for a fleeting escape leads to an unsettling and endless suspension of existence']

## Image Generation

In [60]:
import replicate

In [61]:
client = replicate.Client(api_token=os.environ["STABLEDIFFUSION_API_KEY"])

In [62]:
def generate_stablediffusion_image(image_prompt):
    output = client.run(
        "stability-ai/stable-diffusion:ac732df83cea7fff18b8472768c88ad041fa750ff7682a21affe81863cbe77e4",
        input = {
            "prompt": image_prompt,
            "height": 1024,
            "width": 576,
            "num_outputs": 1,
            "num_inference_steps": 50,
            "guidance_scale": 7.5,
            "scheduler": 'DPMSolverMultistep',
            "seed": 42
        }
    )
    return output[0]

In [63]:
generated_images = [generate_stablediffusion_image(prompt) for prompt in image_prompts]
generated_images

['https://pbxt.replicate.delivery/lktKofrebunw2UMU7fwgNO9lx85jJepLHmZs7fEaSSURYc4LC/out-0.png',
 'https://pbxt.replicate.delivery/QtG59lznUtbLLZnegrxr38rgyjvge2nrSHMU9iuEAxUXjDfiA/out-0.png',
 'https://pbxt.replicate.delivery/iAcQylyDJO6VHpnnfHqHEP9OBuYVJBYescCmbz0bdjJbjDfiA/out-0.png',
 'https://pbxt.replicate.delivery/rmMyr9oFjTrEO14jTITprfz8XbIGMizyBnOZeFPmgvsfGHeFB/out-0.png',
 'https://pbxt.replicate.delivery/Zgxu8hqKYHLsBpqjmzf7eKNJ8fIV0ZjQhT3StvRmggLGHHeFB/out-0.png',
 'https://pbxt.replicate.delivery/RfuzfsAFHkjPIEPA8Hf38mYemMede9EXdSkKwjFD6plc64wXE/out-0.png']

In [64]:
import urllib.request

# Set the directory path to save the images
directory_path = "/content/drive/MyDrive/Colab Notebooks/UD/img"

# Create the directory if it does not exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Save the images to the folder
for i, image_url in enumerate(generated_images):
    file_path = f"{directory_path}/image_{i+1}.jpg"
    urllib.request.urlretrieve(image_url, file_path)
    print(f"Image {i+1} saved successfully at {file_path}.")

Image 1 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/img/image_1.jpg.
Image 2 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/img/image_2.jpg.
Image 3 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/img/image_3.jpg.
Image 4 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/img/image_4.jpg.
Image 5 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/img/image_5.jpg.
Image 6 saved successfully at /content/drive/MyDrive/Colab Notebooks/UD/img/image_6.jpg.


## Audio Generation

In [65]:
from elevenlabs import set_api_key, generate, play

In [66]:
set_api_key(os.environ["ELEVENLABS_API_KEY"])

In [67]:
# Generate audio using 'elevenlabs' package
audio = generate(
    text=preprocessed_content,
    voice="Nicole",
    model="eleven_monolingual_v1"
)

# Set the directory path to save the images
directory_path = "/content/drive/MyDrive/Colab Notebooks/UD/aud"

# Create the directory if it does not exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

audio_file_path = os.path.join(directory_path, f"audio_0.wav")

# Save the generated audio to the temporary directory
with open(audio_file_path, 'wb') as f:
    f.write(audio)

## Compositing

In [68]:
from moviepy.editor import ImageClip, concatenate_videoclips
from moviepy.editor import AudioFileClip

# Paths to images and audio
images_folder = '/content/drive/MyDrive/Colab Notebooks/UD/img/'
audio_file_path = '/content/drive/MyDrive/Colab Notebooks/UD/aud/audio_0.wav'

# Get image files from the folder
image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]
# Sort image files by name
image_files.sort()
# Create a list of image paths
image_paths = [os.path.join(images_folder, f) for f in image_files]

# Load audio clip
audio_clip = AudioFileClip(audio_file_path)

# Calculate duration per image
duration_per_image = audio_clip.duration / len(image_paths)

# Create video clips for each image
video_clips = []
for image_path in image_paths:
    img_clip = ImageClip(image_path, duration=duration_per_image).resize((1080, 1920))
    video_clips.append(img_clip)

# Concatenate video clips
concatenated_clips = concatenate_videoclips(video_clips, method="compose")
# Set audio for the concatenated video
final_clip = concatenated_clips.set_audio(audio_clip)
# Set the FPS for the final video
final_clip.fps = 24

# Set the directory path to save the images
directory_path = "/content/drive/MyDrive/Colab Notebooks/UD/output"
# Create the directory if it does not exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Export the video
formatted_timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
output_video_path = f'/content/drive/MyDrive/Colab Notebooks/UD/output/output_{formatted_timestamp}_reddit.mp4'
final_clip.write_videofile(output_video_path, codec='libx264', threads=4)

Moviepy - Building video /content/drive/MyDrive/Colab Notebooks/UD/output/output_20230830000321_reddit.mp4.
MoviePy - Writing audio in output_20230830000321_redditTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/Colab Notebooks/UD/output/output_20230830000321_reddit.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Colab Notebooks/UD/output/output_20230830000321_reddit.mp4


## Uploading

In [69]:
import google.auth
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

In [70]:
# Load authorized user info from file
with open('/content/drive/MyDrive/Colab Notebooks/UD/token.json', 'r') as file:
    authorized_user_info = json.load(file)

creds = Credentials.from_authorized_user_info(authorized_user_info, ['https://www.googleapis.com/auth/youtube.upload'])
creds.refresh(Request())
credentials = creds

youtube = googleapiclient.discovery.build('youtube', 'v3', credentials=credentials)

In [73]:
# Define the function to upload the video file
def upload_short(video_title, video_description, video_file_path):
    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {
                "title": video_title,
                "description": video_description,
                "defaultLanguage": "en",
                "defaultAudioLanguage": "en"
            },
            "status": {
                "privacyStatus": "private",
                "madeForKids": False,
                "selfDeclaredMadeForKids": False,
                "embeddable": True,
                "license": "youtube",
                "publicStatsViewable": True
            },
        },
        media_body=googleapiclient.http.MediaFileUpload(video_file_path),
        restricts=[{
            "restriction": "label",
            "restrictionText": "No restrictions"
        }]
    )
    response = request.execute()
    return response

In [72]:
# Call the upload function with your desired parameters
response = upload_short(shorts_title, shorts_description, output_video_path)
response

{'kind': 'youtube#video',
 'etag': 'fQCXo2S1H5R1Ee9u6hJpwQ5e2QU',
 'id': '-34fKb3k79M',
 'snippet': {'publishedAt': '2023-08-30T00:07:10Z',
  'channelId': 'UCWdV2EMmhOMenikQpuFdWog',
  'title': 'YouTube Short: The Mystery Naps and the Unseen Reality',
  'description': '"Escaping Reality: A YouTube Short about a Woman\'s Mysterious Disappearing Act"',
  'thumbnails': {'default': {'url': 'https://i9.ytimg.com/vi/-34fKb3k79M/default.jpg?sqp=CKyPuqcG&rs=AOn4CLDpUO4MVwlWy_FDE9IRc3zh-vCRQg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i9.ytimg.com/vi/-34fKb3k79M/mqdefault.jpg?sqp=CKyPuqcG&rs=AOn4CLDA-75mL5xHCY4bA0OFxTgz06thMQ',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i9.ytimg.com/vi/-34fKb3k79M/hqdefault.jpg?sqp=CKyPuqcG&rs=AOn4CLCLC2ATcuYZpb2-ked3s9XmpvNHYQ',
    'width': 480,
    'height': 360}},
  'channelTitle': 'Nikhil Nair',
  'categoryId': '22',
  'liveBroadcastContent': 'none',
  'localized': {'title': 'YouTube Short: The Mystery Naps 